# Tarea 6 - Extracción de Características

- Martínez Ostoa Néstor
- Minería de Textos
- IIMAS, UNAM

## Preparación de datos

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

print(spam_data.shape)

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

(5572, 2)


,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


## División de muestras en entrenamiento (train) y prueba (test)

In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)
print(len(X_train))

4179


### Pregunta 1

Ajustar los datos de entrenamiento `X_train` utilizando un `count_vectorizer` con parámetros predeterminados.

¿Cuál es el token más largo en el vocabulario?

*Esta función debería devolver una cadena.*

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

def respuesta_uno (X_train):
    vectorizer = CountVectorizer(lowercase=True)
    vectorizer.fit_transform(X_train)
    
    largest_token = ""
    for token in vectorizer.get_feature_names_out():
        if len(token) > len(largest_token):
            largest_token = token
            
    return largest_token

respuesta_uno(X_train)

'com1win150ppmx3age16subscription'

In [4]:
lt = ""
for i in range(X_train.shape[0]):
    text = X_train.iloc[i]
    text = text.split()
    for w in text:
        if len(w) > len(lt):
            lt = w
            
print(lt)

88039.SkilGme.TsCs087147403231Winawk!Age16+å£1.50perWKsub


### Pregunta 2

¿Cuál es el número promedio de caracteres por documento para los documentos no spam y spam?

*Esta función debe devolver una tupla (promedio de # caracteres no es spam, promedio # caracteres spam).*

In [5]:
def avg_token(text):
    lengths = []
    for word in text:
        lengths.append(len(word))

    lengths = np.array(lengths)
    return np.round(np.sum(lengths) / lengths.shape[0], 4)

def respuesta_dos(spam_data):
    X_spam = spam_data[spam_data['target'] == 1]
    X_ham = spam_data[spam_data['target'] == 0]
    
    assert X_spam['target'].unique()[0] == 1
    assert X_ham['target'].unique()[0] == 0
    
    return (avg_token(X_spam['text']), avg_token(X_ham['text']))

respuesta_dos(spam_data)

(138.8661, 71.0236)

### Pregunta 3

¿Cuál es el número promedio de dígitos por documento para los documentos no spam y spam?

*Esta función debe devolver una tupla (promedio de # dígitos no es spam, promedio # dígitos spam).*

In [6]:
def avg_digit(text):
    num_docs = text.shape[0]
    digits_per_doc = []
    
    for word in text:
        digits = 0
        for c in word:
            if c.isdigit():
                digits += 1
        digits_per_doc.append(digits)
    
    return np.round(np.sum(np.array(digits_per_doc)) / num_docs, 4)


def respuesta_tres(spam_data):
    X_spam = spam_data[spam_data['target'] == 1]
    X_ham = spam_data[spam_data['target'] == 0]
    
    assert X_spam['target'].unique()[0] == 1
    assert X_ham['target'].unique()[0] == 0
    
    return (avg_digit(X_spam['text']), avg_digit(X_ham['text']))
    
respuesta_tres(spam_data)

(15.759, 0.2993)

### Pregunta 4

¿Cuál es el número promedio de caracteres que no son palabras (cualquier cosa que no sea una letra, un dígito o un guión bajo) por documento para los documentos que no son spam y spam?

*Sugerencia: utilice las clases de caracteres `\ w` y` \ W`*

*Esta función debe devolver una tupla (promedio de # caracteres que no son palabras, no spam, promedio de # caracteres que no son palabras, spam).*

In [7]:
import re 

def pregunta_cuatro(spam_data):
    temp = spam_data.copy()
    temp['non_words'] = temp['text'].apply(lambda x: len(re.findall(r'\W', x)))
    
    return (
        temp[temp.target == 0]['non_words'].mean(), 
        temp[temp.target == 1]['non_words'].mean()
    )

pregunta_cuatro(spam_data)

(17.29181347150259, 29.041499330655956)

### Pregunta 5

¿Cuál es el tamaño del vocabulario en `X_train` y `X_test`, primero utilizando la función `fit_transform` en ambos (train y test), luego utilizando `fit_transform` sobre el train y solo `transform` en el test



*Esta función debe devolver dos tuplas una con `fit_transform` y la otra con `transform` (vocabulario en `X_train`, vocabulario en `X_test`), (vocabulario en `X_train`, vocabulario en `X_test`).*.*

In [8]:
def respuesta_cinco(X_train, X_test):
    cv1 = CountVectorizer()
    X1 = cv1.fit_transform(X_train)
    cv2 = CountVectorizer()
    X2 = cv2.fit_transform(X_test)
    
    cv3 = CountVectorizer()
    X3 = cv3.fit_transform(X_train)
    X4 = cv3.transform(X_test)
    
    return (X1.shape[1], X2.shape[1], X3.shape[1], X4.shape[1])
    
respuesta_cinco(X_train, X_test)

(7354, 4170, 7354, 7354)

### Pregunta 6

¿Cuales son las 10 palabras mas frecuentes (sin tener en cuenta *Stopwords*) en los documentos que no son spam y spam?


*Esta función debe devolver una tupla (palabras mas frecuentes, no spam, palabras mas frecuentes, spam).*

In [9]:
from nltk.corpus import stopwords
from collections import Counter

stop_words = set(stopwords.words('english'))

def most_common(spam_data, target, k):
    d = '\n'.join(spam_data[spam_data['target'] == target]['text'].values).split()
    c = Counter(d)
    return [t[0] for t in c.most_common(k)]

def pregunta_seis(spam_data, stop_words, k=10):
    spam_data = spam_data.copy()
    spam_data['text'] = spam_data['text'].apply(lambda x : ' '.join(w.lower() for w in x.split() if w not in stop_words and len(w) > 2))
    
    return (most_common(spam_data, 0, k), most_common(spam_data, 1, k))

pregunta_seis(spam_data, stop_words)

(["i'm",
  'get',
  '&lt;#&gt;',
  'got',
  'like',
  'call',
  'come',
  'know',
  'you',
  'good'],
 ['call',
  'free',
  'txt',
  'text',
  'mobile',
  'claim',
  'reply',
  'stop',
  'you',
  'get'])

### Pregunta 7

¿Cuales son las 10 palabras mas frecuentes (solo teniendo en cuenta *Stopwords*) en los documentos que no son spam y spam?


*Esta función debe devolver una tupla (palabras mas frecuentes, no spam, palabras mas frecuentes, spam).*

In [10]:
def pregunta_siete(spam_data, stop_words, k=10):
    spam_data = spam_data.copy()
    spam_data['text'] = spam_data['text'].apply(
        lambda x : ' '.join(
            w for w in x.split() if w in stop_words
        )
    )
    return (most_common(spam_data, 0, k), most_common(spam_data, 1, k))

pregunta_siete(spam_data, stop_words)

(['to', 'you', 'the', 'a', 'and', 'i', 'in', 'is', 'my', 'me'],
 ['to', 'a', 'your', 'or', 'the', 'for', 'you', 'is', 'on', 'have'])